In [ ]:
"""
README :
ce code permet 1) de prétraiter le corpus  : détecter la langue des textes, nettoyer et dédoublonner les textes
               NB.si on ne fait pas correspondre la langue des textes et des labels, les items se convergent très mal !
                    -->listings_en, listings_fr

               2) de faire zéro shot classification sur un GPU,
               le résultat prend id, host_about, labels... comme colonnes.
                    -->bge_zsc_en,bge_zsc_fr

               3) de renvoyer les scores de zéro shot aux listings en fonction de 'host_about' et remplir les nan par 0
                    -->listings_zsc

"""

import os
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from transformers import pipeline
import pandas as pd
import re
import numpy as np
import torch
import math

from transformers import pipeline
import torch._dynamo
torch._dynamo.config.suppress_errors = True

#répertoires pour les résultats
# mettre les scripts dans le dossier "code", listings original dans le dossier 'corpus'

data_folder='../data' #pour listings filtrés
result_folder='../result_airbnb'#pour les résultats de zéro shot
figs_folder='../figs'#pour les figures

os.makedirs(data_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)
os.makedirs(figs_folder, exist_ok=True)


In [ ]:
"""
tester le modèle :
"MoritzLaurer/bge-m3-zeroshot-v2.0"
"""
classifier_bge = pipeline("zero-shot-classification", model="MoritzLaurer/bge-m3-zeroshot-v2.0", use_safetensors=False,device=2)

sequences = [
    "The government passed a new policy.",
    "This dish uses a lot of spices.",
    "She won the tennis championship."
]

labels = ["politics", "cooking", "sports"]

results = classifier_bge(sequences, candidate_labels=labels, multi_label=True)

Device set to use cuda:2
/home/YLiu/.venv/lib64/python3.9/site-packages/torch/utils/data/dataloader.py:643: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f090fe50580> was reported to be 3 (when accessing len(dataloader)), but 4 samples have been fetched. 
  warnings.warn(warn_msg)
/home/YLiu/.venv/lib64/python3.9/site-packages/torch/utils/data/dataloader.py:643: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f090fe50580> was reported to be 3 (when accessing len(dataloader)), but 5 samples have been fetched. 
  warnings.warn(warn_msg)
/home/YLiu/.venv/lib64/python3.9/site-packages/torch/utils/data/dataloader.py:643: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f090fe50580> was reported to be 3 (when accessing len(dataloader)), but 6 samples have been fetched. 
  warnings.warn(warn_msg)
/home/YLiu/.venv/lib6

# prétraitement de corpus:

In [ ]:
#charger le corpus
listings=pd.read_csv("corpus/listings.csv")#changer le répertoire
print(len(listings))
print(listings.columns)
display(listings.head())

74329
Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3109,https://www.airbnb.com/rooms/3109,20231212042736,2023-12-12,city scrape,Rental unit in Paris · ★5.0 · 1 bedroom · 1 be...,NaN,Good restaurants<br />very close the Montparna...,https://a0.muscache.com/pictures/baeae9e2-cd53...,3631,...,5.00,5.00,5.00,7511409139079,f,1,1,0,0,0.05
1,5396,https://www.airbnb.com/rooms/5396,20231212042736,2023-12-14,city scrape,Rental unit in Paris · ★4.59 · Studio · 1 bed ...,NaN,"You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903,...,4.84,4.95,4.56,7510402838018,f,2,1,1,0,2.12
2,81106,https://www.airbnb.com/rooms/81106,20231212042736,2023-12-13,city scrape,Rental unit in Paris · ★4.84 · 1 bedroom · 1 b...,NaN,The neighborhood will show you an other side o...,https://a0.muscache.com/pictures/miso/Hosting-...,439130,...,4.89,4.36,4.83,7511006045681,f,1,1,0,0,0.43
3,7397,https://www.airbnb.com/rooms/7397,20231212042736,2023-12-13,city scrape,Rental unit in Paris · ★4.73 · 2 bedrooms · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/67928287/330b...,2626,...,4.88,4.93,4.73,7510400829623,f,7,7,0,0,2.22
4,7964,https://www.airbnb.com/rooms/7964,20231212042736,2023-12-12,city scrape,Rental unit in Paris · ★4.80 · 1 bedroom · 1 b...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,22155,...,5.00,5.00,5.00,7510903576564,f,1,1,0,0,0.03


In [ ]:
#détecter la langue :

from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
import pandas as pd
DetectorFactory.seed = 0

def detect_language(text):
    # classer les textes en fr/en/other_lang/nan s'il n'y a pas de textes
    try:
        if pd.isna(text) or str(text).strip() == "":
            return None
        lang = detect(str(text))
        if lang in ["en", "fr"]:
            return lang
        else:
            return "other_lang"
    except LangDetectException:
        return None

listings["langue"] = listings["host_about"].apply(detect_language)
listings.to_csv('../data/listings.csv', index=False)

In [ ]:
print(listings.langue.value_counts()/len(listings))#exclu nan

langue
fr            0.246404
en            0.179795
other_lang    0.015135
Name: count, dtype: float64


In [ ]:
def is_valid_text(text):
    """
    filtrer les textes:
        pas de nan,
        pas que les ponctuation,
        pas que les liens url
        pas que les caractères
    renvoyer T/F
    """
        if pd.isna(text):  # ==dropnaNaN
            return False
        text = str(text).strip()
        if len(text) == 0:  # strip
            return False
        # effacer ce qui n'a que de ponctuation
        if not re.search(r"[A-Za-zÀ-ÖØ-öø-ÿ]", text):
            return False
         # effacer url
        if re.fullmatch(r"https://\S+", text):
            return False
        return True


#1) dropna, drop duplicates: (reste textes uniques)
sub_listings=listings.dropna(subset='host_about').drop_duplicates(subset='host_about')
print(len(sub_listings))

#2) enlever les textes irrégulier
sub_listings=sub_listings[sub_listings['host_about'].apply(is_valid_text)].copy()


#3) prend que les textes fr et en :(optionnel)
# # ou séparer les textes fr et non-fr
# sub_listings=sub_listings[(sub_listings['langue']=='fr')|(sub_listings['langue']=='en')]
print(sub_listings.langue.value_counts())
print(f"listings initial: {len(listings)}, après filtrage: {len(sub_listings)}")


20882
langue
fr    12109
en     7878
Name: count, dtype: int64
listings initial: 74329, après filtrage: 19987


In [ ]:
#sauvegarder séparément:
listings_fr=sub_listings[sub_listings['langue']=='fr']
listings_fr.to_csv('../data/listings_fr.csv',index=False)#changer de chemin, si besoin
print(len(listings_fr))

listings_en=sub_listings[sub_listings['langue']!='fr']
listings_en.to_csv('../data/listings_en.csv',index=False)
print(len(listings_en))


12109
7878


# zéro shot classification:

In [ ]:
def zsc_gpu(
    corpus_path,
    result_path,
    candidate_labels,
    model_name="tasksource/ModernBERT-large-nli",
    device=2,
    save_interval=1000
    ):

    #------------------assurer que le dossier de result_path exist
    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    #------------------charger le corpus
    corpus=pd.read_csv(corpus_path)
    # df = corpus[:50]
    df=corpus.copy()

    #------------------charger le modèle
    classifier = pipeline(
        "zero-shot-classification",
        model=model_name,
        device=device
    )
    print(f"[MODEL] {model_name} loaded!")


    #---------------zéro shot classification ligne par ligne
    #---------------renvoir id, host_about, labels....
    def classify_row(row):
        text = row["host_about"]
        try:
            res = classifier(text, candidate_labels, multi_label=True)#***
            return {**{'id': row['id'], 'host_about': text},
                    **dict(zip(res['labels'], res['scores']))}
        except Exception:
            return {**{'id': row['id'], 'host_about': text},
                    **{label: np.nan for label in candidate_labels}}


    #commencer!
    results_list = []
    start_time = time.time()
    print(f"[START] ZSC completed on {len(df)} paragraphs with {len(candidate_labels)} labels...")

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Zero-shot classification"):
        results_list.append(classify_row(row))
        # sauvegarder tous les 'save_interval' lignes
        if (idx + 1) % save_interval == 0:
            pd.DataFrame(results_list).to_csv(result_path, index=False)
            print(f"[SAVE] Progress saved at {idx+1} / {len(df)} rows")
    # sauvegarder le dernier échantillon
    if results_list:
        pd.DataFrame(results_list).to_csv(result_path, index=False)
    end_time = time.time()


    print(f"résultat sauvegardé dans f{result_path}!")
    print(f"[SUCCESS] ZSC sur {len(df)} textes avec {len(candidate_labels)} labels  par {model_name} prend {end_time - start_time:.2f} sec !")
    return pd.DataFrame(results_list)


In [ ]:
## EXEMPLE :
# labels=[
#     'open to different cultures', 'cosmopolitan','international view', 'cultural exchange',
#     'personal life', 'life experiences', 'divers interests', 'hobbies', 'enjoy life',
#     'meet new people', 'welcoming', 'friendly', 'sociable', 'interpersonal interaction',
#     'thoughtful service', 'attentive to needs', 'willing to help', 'responsive',
#     'fan of Airbnb', 'Airbnb community','love Airbnb', 'travel with Airbnb'
# ]

# airbnb_zsc= zsc_gpu(
#     corpus_path="../corpus/listings",
#     result_path="../result_airbnb/bge_zsc_en.csv",
#     candidate_labels=labels,
#     model_name="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
#     device=2,
#     save_interval=1000
#     )

# airbnb_zsc.head()

Device set to use cuda:2


[MODEL] tasksource/ModernBERT-large-nli loaded!
[START] ZSC completed on 50 paragraphs with 22 labels...


Zero-shot classification: 100%|██████████| 50/50 [00:16<00:00,  2.95it/s]

résultat sauvegardé dans fresult_airbnb/zsc_final.csv
[SUCCESS] ZSC completed on 50 paragraphs with 22 labels in 16.96 sec


,id,host_about,thoughtful service,welcoming,friendly,attentive to needs,life experiences,interpersonal interaction,open to different cultures,enjoy life,...,cosmopolitan,love Airbnb,divers interests,fan of Airbnb,responsive,meet new people,Airbnb community,personal life,travel with Airbnb,hobbies
0,5396,We have spent a lot of time traveling for work...,0.988275,0.978899,0.969900,0.969503,0.940130,0.923169,0.801536,0.758241,...,0.348484,0.312117,0.218350,0.176688,0.154070,0.122186,0.093593,0.084991,0.076266,0.061907
1,81106,Independent photographer. \nCyclist. Often on ...,0.920913,0.964100,0.934329,0.729547,0.964321,0.889373,0.890125,0.906853,...,0.523540,0.840635,0.962668,0.897780,0.399496,0.675189,0.381324,0.966522,0.443091,0.954231
2,7397,"I am a writer,54, author of novels, books of l...",0.028080,0.031486,0.065039,0.036001,0.907861,0.072091,0.060694,0.030317,...,0.291079,0.004195,0.958066,0.017159,0.019739,0.003472,0.002388,0.946889,0.002107,0.552791
3,7964,Hello ! \r\nOur apartment is great and I am su...,0.609405,0.979261,0.929709,0.287556,0.150899,0.798147,0.115259,0.668866,...,0.159512,0.814343,0.134178,0.467341,0.306458,0.242886,0.459811,0.475780,0.378387,0.049576
4,81615,We are web startup entrepreneurs and bnb/garde...,0.195595,0.635350,0.536704,0.215333,0.962923,0.603187,0.703285,0.972081,...,0.825853,0.976903,0.969104,0.981594,0.260909,0.242315,0.887706,0.916966,0.867488,0.495915


# zsc selon langue :

In [ ]:
labels_en=[
    'open to different cultures', 'cosmopolitan','international view', 'cultural exchange',
    'personal life', 'life experiences', 'divers interests', 'hobbies', 'enjoy life',
    'meet new people', 'welcoming', 'friendly', 'sociable', 'interpersonal interaction',
    'thoughtful service', 'attentive to needs', 'willing to help', 'responsive',
    'fan of Airbnb', 'Airbnb community','love Airbnb', 'travel with Airbnb'
]

airbnb_zsc=zsc_gpu(
    corpus_path='../data/listings_en.csv',
    result_path='../result_airbnb/bge_zsc_en.csv',
    candidate_labels=labels_en,
    model_name="MoritzLaurer/bge-m3-zeroshot-v2.0",
    device=2,
    save_interval=1000
    )


airbnb_zsc.head()
#[SUCCESS] ZSC completed on 7879 paragraphs with 22 labels  by MoritzLaurer/bge-m3-zeroshot-v2.0 in 1466.57 sec

Device set to use cuda:2


[MODEL] MoritzLaurer/bge-m3-zeroshot-v2.0 loaded!
[START] ZSC completed on 7879 paragraphs with 22 labels...


Zero-shot classification:  13%|█▎        | 1000/7879 [03:13<21:23,  5.36it/s]

[SAVE] Progress saved at 1000 / 7879 rows


Zero-shot classification:  25%|██▌       | 2000/7879 [06:21<18:31,  5.29it/s]

...

Zero-shot classification:  89%|████████▉ | 7001/7879 [21:47<03:15,  4.50it/s]

[SAVE] Progress saved at 7000 / 7879 rows


Zero-shot classification: 100%|██████████| 7879/7879 [24:26<00:00,  5.37it/s]


résultat sauvegardé dans fresult_airbnb/bge_zsc_en.csv
[SUCCESS] ZSC completed on 7879 paragraphs with 22 labels  by MoritzLaurer/bge-m3-zeroshot-v2.0 in 1466.57 sec


,id,host_about,attentive to needs,welcoming,friendly,sociable,responsive,thoughtful service,international view,interpersonal interaction,...,enjoy life,cultural exchange,divers interests,open to different cultures,travel with Airbnb,fan of Airbnb,love Airbnb,meet new people,personal life,hobbies
0,5396.0,We have spent a lot of time traveling for work...,0.995195,0.986460,0.982835,0.982809,0.973056,0.970217,0.957682,0.953720,...,0.721678,0.588577,0.508959,0.326977,0.312311,0.285508,0.137775,0.111350,0.010742,0.008257
1,81106.0,Independent photographer. \nCyclist. Often on ...,0.811337,0.806805,0.141630,0.584475,0.639749,0.706699,0.766916,0.751882,...,0.712022,0.001054,0.950056,0.270624,0.258570,0.095681,0.021362,0.011969,0.831021,0.879059
2,7397.0,"I am a writer,54, author of novels, books of l...",0.103298,0.073750,0.029494,0.013494,0.380097,0.147684,0.025538,0.002643,...,0.001584,0.012215,0.862305,0.190189,0.001214,0.000321,0.000308,0.000565,0.015806,0.063168
3,7964.0,Hello ! \r\nOur apartment is great and I am su...,0.816022,0.996871,0.993857,0.951562,0.535471,0.842520,0.305249,0.996420,...,0.178647,0.040200,0.174614,0.395662,0.692560,0.294100,0.050386,0.825821,0.251737,0.000458
4,81615.0,We are web startup entrepreneurs and bnb/garde...,0.369725,0.607601,0.237928,0.689847,0.456869,0.445728,0.800902,0.276564,...,0.910597,0.027181,0.943947,0.510266,0.185741,0.541624,0.281515,0.026181,0.606292,0.045909


In [ ]:
labels_fr=[
        'ouvert aux différentes cultures', 'cosmopolite','vue internationale', 'échange culturel',
        'vie personnelle', 'parcours personnel', 'loisirs', 'passions', 'aimer la vie',
       'rencontrer de nouvelles personnes', 'accueillant', 'amical','sociable', 'interaction interpersonnelle',
       'rendre service','attentif aux besoins', 'prêt à aider', 'réactif',
       "adepte d'Airbnb",'communauté Airbnb', 'aime Airbnb', 'voyager par Airbnb'
       ]
airbnb_zsc=zsc_gpu(
    corpus_path='data/listings_fr.csv',
    result_path='result_airbnb/bge_zsc_fr.csv',
    candidate_labels=labels_fr,
    model_name="MoritzLaurer/bge-m3-zeroshot-v2.0",
    device=3,
    save_interval=1000
    )
airbnb_zsc.head()
# [SUCCESS] ZSC completed on 12109 paragraphs with 22 labels  by MoritzLaurer/bge-m3-zeroshot-v2.0 in 2321.55 sec


# unifier labels et concaténer les résultats：

In [ ]:
import pandas as pd
import os
import time
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


labels_en=[
    'open to different cultures', 'cosmopolitan','international view', 'cultural exchange',
    'personal life', 'life experiences', 'divers interests', 'hobbies', 'enjoy life',
    'meet new people', 'welcoming', 'friendly', 'sociable', 'interpersonal interaction',
    'thoughtful service', 'attentive to needs', 'willing to help', 'responsive',
    'fan of Airbnb', 'Airbnb community','love Airbnb', 'travel with Airbnb'
]

labels_fr=[
        'ouvert aux différentes cultures', 'cosmopolite','vue internationale', 'échange culturel',
        'vie personnelle', 'parcours personnel', 'loisirs', 'passions', 'aimer la vie',
       'rencontrer de nouvelles personnes', 'accueillant', 'amical','sociable', 'interaction interpersonnelle',
       'rendre service','attentif aux besoins', 'prêt à aider', 'réactif',
       "adepte d'Airbnb",'communauté Airbnb', 'aime Airbnb', 'voyager par Airbnb'
       ]
fr2en=dict(zip(labels_fr, labels_en))
print(fr2en)

{'ouvert aux différentes cultures': 'open to different cultures', 'cosmopolite': 'cosmopolitan', 'vue internationale': 'international view', 'échange culturel': 'cultural exchange', 'vie personnelle': 'personal life', 'parcours personnel': 'life experiences', 'loisirs': 'divers interests', 'passions': 'hobbies', 'aimer la vie': 'enjoy life', 'rencontrer de nouvelles personnes': 'meet new people', 'accueillant': 'welcoming', 'amical': 'friendly', 'sociable': 'sociable', 'interaction interpersonnelle': 'interpersonal interaction', 'rendre service': 'thoughtful service', 'attentif aux besoins': 'attentive to needs', 'prêt à aider': 'willing to help', 'réactif': 'responsive', "adepte d'Airbnb": 'fan of Airbnb', 'communauté Airbnb': 'Airbnb community', 'aime Airbnb': 'love Airbnb', 'voyager par Airbnb': 'travel with Airbnb'}


In [ ]:
#charger les listings et lesrésultats:
listings=pd.read_csv("../corpus/listings.csv")
print(len(listings))
res_en=pd.read_csv("../result_airbnb/bge_zsc_en.csv")
print(len(res_en))
res_fr=pd.read_csv("../result_airbnb/bge_zsc_fr.csv")
print(len(res_fr))

D:\temp\ipykernel_14344\2850117821.py:2: DtypeWarning: Columns (6,22,41,45,46,61) have mixed types. Specify dtype option on import or set low_memory=False.
  listings=pd.read_csv("../corpus/listings.csv")


74330
7879
12109


In [ ]:
#renommer:
res_fr=res_fr.rename(columns=fr2en)
res_fr.columns

#concaténer:
res=pd.concat([res_en,res_fr], axis=0)
order = [
    "id", "host_about",
    "open to different cultures", "cosmopolitan", "international view", "cultural exchange",
    "personal life", "life experiences", "divers interests", "hobbies", "enjoy life",
    "meet new people", "welcoming", "friendly", "sociable", "interpersonal interaction",
    "thoughtful service", "attentive to needs", "willing to help", "responsive",
    "fan of Airbnb", "Airbnb community", "love Airbnb", "travel with Airbnb"
]
res = res[order]
print(len(res))#==len(res_en)+len(res_fr)


#renvoyer les scores aux listings originaux
res=res.drop(columns='id')
listings_zsc=listings.merge(res, left_on='host_about', right_on='host_about', how='left')
listings_zsc[labels_en]=listings_zsc[labels_en].fillna(0)#pour les lignes qui n'ont pas 'host_about'
display(listings_zsc.head())

## sauvegarder
listings_zsc.to_csv("../result_airbnb/listings_zsc.csv")

19988


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,sociable,interpersonal interaction,thoughtful service,attentive to needs,willing to help,responsive,fan of Airbnb,Airbnb community,love Airbnb,travel with Airbnb
0,3109.0,https://www.airbnb.com/rooms/3109,2.023121e+13,2023-12-12,city scrape,Rental unit in Paris · ★5.0 · 1 bedroom · 1 be...,NaN,Good restaurants<br />very close the Montparna...,https://a0.muscache.com/pictures/baeae9e2-cd53...,3631.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,5396.0,https://www.airbnb.com/rooms/5396,2.023121e+13,2023-12-14,city scrape,Rental unit in Paris · ★4.59 · Studio · 1 bed ...,NaN,"You are within walking distance to the Louvre,...",https://a0.muscache.com/pictures/52413/f9bf76f...,7903.0,...,0.982809,0.953720,0.970217,0.995195,0.911915,0.973056,0.285508,0.749040,0.137775,0.312311
2,81106.0,https://www.airbnb.com/rooms/81106,2.023121e+13,2023-12-13,city scrape,Rental unit in Paris · ★4.84 · 1 bedroom · 1 b...,NaN,The neighborhood will show you an other side o...,https://a0.muscache.com/pictures/miso/Hosting-...,439130.0,...,0.584475,0.751882,0.706699,0.811337,0.294656,0.639749,0.095681,0.093284,0.021362,0.258570
3,7397.0,https://www.airbnb.com/rooms/7397,2.023121e+13,2023-12-13,city scrape,Rental unit in Paris · ★4.73 · 2 bedrooms · 2 ...,NaN,NaN,https://a0.muscache.com/pictures/67928287/330b...,2626.0,...,0.013494,0.002643,0.147684,0.103298,0.022032,0.380097,0.000321,0.000877,0.000308,0.001214
4,7964.0,https://www.airbnb.com/rooms/7964,2.023121e+13,2023-12-12,city scrape,Rental unit in Paris · ★4.80 · 1 bedroom · 1 b...,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,22155.0,...,0.951562,0.996420,0.842520,0.816022,0.983836,0.535471,0.294100,0.898153,0.050386,0.692560
